In [9]:
# from tensorflow.keras.appliications import EfficientNetB0
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.metrics import AUC,Precision,Recall
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import tensorflow as tf
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,Tensorboard

from tensorflow.keras.mixed_precision import set_global_policy
import numpy as np
import os

gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu,True)
    except RuntimeError as e:
        print(e)

In [ ]:
# # https://stackoverflow.com/questions/51995977/how-can-i-use-a-pre-trained-neural-network-with-grayscale-images
# base_model = EfficientNetB0(include_top=False,weights = 'imagenet')

# inputs = layers.Input(shape=(224,224,1))
# x = layers.Conv2D(3,(3,3), padding = 'same')(inputs)
# x = base_model(x)
# x = layers.GlobalAveragePooling2D()(x)
# outputs = layers.Dense(8,activation='sigmoid')(x)

# model = models.Model(inputs,outputs)
# model.summary()

In [2]:
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

# Use mixed precision
# https://keras.io/api/mixed_precision/
set_global_policy('mixed_float16')

# Clear GPU memory
tf.keras.backend.clear_session()


with tf.device('/GPU:0'):
    base_model = EfficientNetB0(include_top=False,weights = 'imagenet')

    inputs = layers.Input(shape=(224,224,1))
    x = layers.Conv2D(3,(3,3), padding = 'same')(inputs)
    # x = layers.Lambda(lambda x: tf.repeat(x, 3, axis=-1))(inputs)
    x = base_model(x)
    x = layers.GlobalAveragePooling2D()(x)
    # x = layers.GlobalAveragePooling2D()(base_model.output)

    # base_model.
    outputs = layers.Dense(7,activation='sigmoid')(x)
    for layer in base_model.layers[:-119]:
        layer.trainable = False


    # model.add

    model = models.Model(inputs,outputs)

    

model.summary()


I0000 00:00:1746125894.784073    1125 gpu_process_state.cc:201] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1746125894.786688    1125 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2050, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 3)    │            30 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 7)              │         8,967 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,058,568 (15.48 MB)

 Trainable params: 3,707,885 (14.14 MB)

 Non-trainable params: 350,683 (1.34 MB)

In [ ]:
print(len(base_model.layers[-119:]))

In [6]:

model.compile(optimizer=Adam(learning_rate=1e-5), loss = "binary_crossentropy", metrics = ["binary_accuracy"])


In [ ]:
# model.compile(optimizer="adam",loss="binary_crossentropy", metrics = ["binary_accuracy"
#                                                                       # , "val_accuracy"
#                                                                     #   ,AUC(), Precision(),Recall()
#                                                                       ])


In [ ]:
# import os
# os.path.exists('/home/aifablab/projects/ElijahKun/TestOne/IS_Report/data/numpy/Y_train.npy')


In [4]:

X_train = np.load("../data/numpy/X_train.npy")
Y_train = np.load("../data/numpy/Y_train.npy")

X_val = np.load("../data/numpy/X_valid.npy")
Y_val = np.load("../data/numpy/Y_valid.npy")


# X_train = np.load("/home/aifablab/projects/ElijahKun/TestOne/IS_Report/data/numpy/X_train.npy")
# Y_train = np.load("/home/aifablab/projects/ElijahKun/TestOne/IS_Report/data/numpy/Y_train.npy")

# X_val = np.load("/home/aifablab/projects/ElijahKun/TestOne/IS_Report/data/numpy/X_valid.npy")
# Y_val = np.load("/home/aifablab/projects/ElijahKun/TestOne/IS_Report/data/numpy/Y_valid.npy")



In [ ]:
print(X_train[0].shape)
print(Y_train.shape)

In [ ]:
my_callbacks = [
    ModelCheckpoint(
        monitor = "val_loss" ,# commonly uses valloss why? cuz secret
        verbose =1,
        filepath = "../checkpoint/model_{epoch:02d}.keras",
        save_best_only = True,
        save_weights_only = False
    )
]

train_ds = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).batch(8)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).batch(8)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    # batch_size = 8,
    verbose = 1,
    callbacks = my_callbacks
)



Epoch 1/15
1574/1574 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - binary_accuracy: 0.8576 - loss: 0.3423
Epoch 1: val_loss improved from inf to 0.33700, saving model to ../checkpoint/model_01.keras
1574/1574 ━━━━━━━━━━━━━━━━━━━━ 189s 88ms/step - binary_accuracy: 0.8576 - loss: 0.3423 - val_binary_accuracy: 0.8582 - val_loss: 0.3370
Epoch 2/15
1574/1574 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - binary_accuracy: 0.8581 - loss: 0.3395
Epoch 2: val_loss improved from 0.33700 to 0.32906, saving model to ../checkpoint/model_02.keras
1574/1574 ━━━━━━━━━━━━━━━━━━━━ 104s 66ms/step - binary_accuracy: 0.8581 - loss: 0.3395 - val_binary_accuracy: 0.8600 - val_loss: 0.3291
Epoch 3/15
1573/1574 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - binary_accuracy: 0.8585 - loss: 0.3384
Epoch 3: val_loss did not improve from 0.32906
1574/1574 ━━━━━━━━━━━━━━━━━━━━ 100s 64ms/step - binary_accuracy: 0.8585 - loss: 0.3384 - val_binary_accuracy: 0.8598 - val_loss: 0.3315
Epoch 4/15
1573/1574 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - binary_acc

In [ ]:
# history = model.fit(
#     train_ds,
#     validation_data = (X_val, Y_val),
#     epochs = 10,
#     batch_size = 8,
#     verbose = 1
#     )

# history = model.fit(
#     train_ds,
#     validation_data=val_ds,
#     epochs=80,
#     # batch_size = 8,
#     verbose = 1,
#     callbacks = my_callbacks
# )


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history["binary_accuracy"])
# plt.plot(history.history["val_accuracy"])
plt.show()

In [ ]:
# X_test = np.load("../data/numpy/X_test.npy")
# Y_test = np.load("../data/numpy/Y_test.npy")
# results = model.evaluate(X_test,Y_test,batch_size=10)



In [ ]:
# predictions = model.predict(X_test[:3])
# print(predictions.shape)
# print(predictions)

In [ ]:
# models.load_model("../checkpoint/model_02.keras")
# print(len(model.layers))
# # for layer in models.layer[:]:


In [ ]:
# continue
my_callbacks = [
    ModelCheckpoint(
        monitor = "val_loss" ,# commonly uses valloss why? cuz secret
        verbose =1,
        filepath = "../checkpoint/model_{epoch:02d}.keras",
        save_best_only = True,
        save_weights_only = False
    )
]

train_ds = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).batch(8)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).batch(8)
model = load_model("../checkpoint/model_14.keras")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    initial_epoch = 14,
    epochs=30,
    # batch_size = 8,
    verbose = 1,
    callbacks = my_callbacks
)

